In [4]:
import argparse
import os
import sys
sys.path.append('C:/Users/arive/pipelinev2')
import matplotlib.pyplot as plt
import cv2
from PIL import Image,ImageDraw

In [5]:
from config.options import Options

from data.dataset import OCRDatasetInstanceSeg
from XML.createXML import XMLData

Unet not supported
Detectron2 not supported
Kraken not supported


In [6]:
cfgfile = "C:/Users/arive/config/config_line.ini"
#cfgfile = "/users/vesalaia/config/config_mask_line.ini"
opts =  Options(cfgfile)

In [7]:
opts.region_classes

{'background': 0, 'content': 1, 'line': 2, 'unknown': 3}

In [8]:
dataset_files = [
    ["C:/Users/arive/Datasets/Muutot/sample_all_printed_1", "C:/Users/arive/Datasets/Muutot/sample_all_printed_1/page"],
    ["C:/Users/arive/Datasets/Muutot/sample_all_printed_2a", "C:/Users/arive/Datasets/Muutot/sample_all_printed_2a/page"],
    ["C:/Users/arive/Datasets/Muutot/sample_all_printed_2b", "C:/Users/arive/Datasets/Muutot/sample_all_printed_2b/page"],
    ["C:/Users/arive/Datasets/Muutot/sample_all_printed_2c", "C:/Users/arive/Datasets/Muutot/sample_all_printed_2c/page"]]

dataset = OCRDatasetInstanceSeg(dataset_files, opts.region_classes)

In [9]:
dataset.__len__()

275

In [10]:
def plot_page_with_regions(image_path, page):
    # Read the image
    thickness = 5
    point_radius = 10
    cell_colour = (0, 0, 255)
    image = cv2.imread(image_path)
    
    if image is None:
        raise ValueError(f"Image not found at the path: {image_path}")
    
    # Convert vertices to a numpy array
#    print(len(page['regions']),page['regions'])
    for reg in page['regions']:
        if reg['regiontype'] == "TableRegion":
            region_colour = (0, 0, 0)
        elif reg['regiontype'] == "TextRegion":
            region_colour = (0, 255, 0)
        
        for cell in reg['table']:
            xmax, ymax = cell['polygon'].max(axis=0)
            xmin, ymin = cell['polygon'].min(axis=0)
            cell_left_top = (xmin, ymin)
            cell_left_bottom = (xmin, ymax)
            cell_right_top = (xmax, ymin)
            cell_right_bottom = (xmax, ymax)
            cv2.circle(image, cell_left_top, radius=point_radius, color=cell_colour)            
            cv2.circle(image, cell_left_bottom, radius=point_radius, color=cell_colour)            
            cv2.circle(image, cell_right_top, radius=point_radius, color=cell_colour)            
            cv2.circle(image, cell_right_bottom, radius=point_radius, color=cell_colour)            
        pts = reg['polygon']
        cv2.polylines(image, [pts], isClosed=True, color=region_colour, thickness=thickness)
            
    # Convert BGR to RGB for displaying with matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Plot the image
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.axis('off')  # Hide axis
    plt.show()


In [1]:
annotations = {'background': 0, 'title': 1, "header": 2, "cell": 3}

def createYOLOsegment(dataset, yolo_path, imgsz=(640,640)):
    if not os.path.exists(yolo_path):
        os.makedirs(yolo_path)
    if not os.path.exists(os.path.join(yolo_path, "images")):
        os.makedirs(os.path.join(yolo_path, "images"))
    if not os.path.exists(os.path.join(yolo_path, "labels")):
        os.makedirs(os.path.join(yolo_path, "labels"))
    for idx in range(dataset.__len__()):
        source, size = dataset.__getsize__(idx)
        destination = f"{yolo_path}/images/img{idx}.jpg"
        img = cv2.imread(source)
        resized_img = cv2.resize(img, imgsz)
        cv2.imwrite(destination, resized_img)
    for idx in range(dataset.__len__()):
        name, size = dataset.__getsize__(idx)
        image_width = size[1]
        image_height = size[0]
        page = dataset.__getXMLitem__(idx)
        if len(page['regions']) > 0:
            file_object = open(f"{yolo_path}/labels/img{idx}.txt", "w")
            regions = []
            for i, reg in enumerate(page['regions']):
                n_reg = {}
                n_reg['index'] = i
                xmax, ymax = reg['polygon'].max(axis=0)
                xmin, ymin = reg['polygon'].min(axis=0)
                n_reg['topcorner'] = (xmin, ymin)
                regions.append(n_reg)
            s_regions = sorted(regions, key=lambda x: (x['topcorner'][1], x['topcorner'][0]), reverse=False)
            if len(page['regions']) == 3:
                category = annotations['title']
                seg = page['regions'][s_regions[0]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                category = annotations['header']
                seg = page['regions'][s_regions[1]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                category = annotations['cell']
                seg = page['regions'][s_regions[2]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")  
            elif len(page['regions']) == 4:
                category = annotations['header']
                seg = page['regions'][s_regions[0]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                seg = page['regions'][s_regions[1]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")
                
                category = annotations['content']
                seg = page['regions'][s_regions[2]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")  
                
                seg = page['regions'][s_regions[3]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")  
            elif len(page['regions']) == 6:
                category = annotations['title']
                seg = page['regions'][s_regions[0]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")
                
                seg = page['regions'][s_regions[1]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    
                
                category = annotations['header']
                seg = page['regions'][s_regions[2]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n") 
                
                seg = page['regions'][s_regions[3]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                category = annotations['content']
                seg = page['regions'][s_regions[4]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")  
                
                seg = page['regions'][s_regions[5]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")  
            else:
                print("Annotation error", idx, dataset.__getname__(idx))
                
            file_object.close()

In [ ]:
yolo_path = "C:/Users/arive/Datasets/Muutot/YOLO"
createYOLOsegment(dataset, yolo_path)

In [23]:
annotations = {'background': 0, 'title': 1, "header": 2, "content": 3}

def createYOLOsegmentRegions(dataset, yolo_path, imgsz=(640,640)):
    if not os.path.exists(yolo_path):
        os.makedirs(yolo_path)
    if not os.path.exists(os.path.join(yolo_path, "images")):
        os.makedirs(os.path.join(yolo_path, "images"))
    if not os.path.exists(os.path.join(yolo_path, "labels")):
        os.makedirs(os.path.join(yolo_path, "labels"))
    for idx in range(dataset.__len__()):
        source, size = dataset.__getsize__(idx)
        destination = f"{yolo_path}/images/img{idx}.jpg"
        img = cv2.imread(source)
        resized_img = cv2.resize(img, imgsz)
        cv2.imwrite(destination, resized_img)
    for idx in range(dataset.__len__()):
        name, size = dataset.__getsize__(idx)
        image_width = size[1]
        image_height = size[0]
        page = dataset.__getXMLitem__(idx)
        if len(page['regions']) > 0:
            file_object = open(f"{yolo_path}/labels/img{idx}.txt", "w")
            regions = []
            for i, reg in enumerate(page['regions']):
                n_reg = {}
                n_reg['index'] = i
                xmax, ymax = reg['polygon'].max(axis=0)
                xmin, ymin = reg['polygon'].min(axis=0)
                n_reg['topcorner'] = (xmin, ymin)
                regions.append(n_reg)
            s_regions = sorted(regions, key=lambda x: (x['topcorner'][1], x['topcorner'][0]), reverse=False)
            if len(page['regions']) == 3:
                category = annotations['title']
                seg = page['regions'][s_regions[0]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                category = annotations['header']
                seg = page['regions'][s_regions[1]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                category = annotations['content']
                seg = page['regions'][s_regions[2]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

            elif len(page['regions']) == 4:
                category = annotations['header']
                seg = page['regions'][s_regions[0]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")    
                
                seg = page['regions'][s_regions[1]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")
                
                category = annotations['content']
                seg = page['regions'][s_regions[2]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                seg = page['regions'][s_regions[3]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

            elif len(page['regions']) == 6:
                category = annotations['title']
                seg = page['regions'][s_regions[0]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")               
                
                seg = page['regions'][s_regions[1]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    
                
                category = annotations['header']
                seg = page['regions'][s_regions[2]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")  
                
                seg = page['regions'][s_regions[3]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                category = annotations['content']
                seg = page['regions'][s_regions[4]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    

                seg = page['regions'][s_regions[5]['index']]['polygon'].flatten().tolist()
                result = [num / image_width if i % 2 == 0 else num /image_height  for i, num in enumerate(seg)]
                reg_coords = ""
                for coord in result:
                    reg_coords = reg_coords + " {}".format(str(coord))
                file_object.write(f"{category} {reg_coords}\n")                    
            else:
                print("Annotation error", idx, dataset.__getname__(idx))
                
            file_object.close()

In [24]:
yolo_path = "C:/Users/arive/Datasets/Muutot/YOLOReg"
createYOLOsegmentRegions(dataset, yolo_path)

Annotation error 145 sample_all_printed_2a\viipuri_muuttaneet_1866-1873_tksrk_mko784-790_122
Annotation error 257 sample_all_printed_2c\keuruu_muuttaneet_1818-1826_ap_9


In [ ]:
page['regions'][3]['table']



In [ ]:
[poly['polygon'] for poly in page['regions']]

In [ ]:
for idx in range(dataset.__len__()):
    page = dataset.__getXMLitem__(idx)
    regions = []
    for i, reg in enumerate(page['regions']):
        n_reg = {}
        n_reg['index'] = i
        xmax, ymax = reg['polygon'].max(axis=0)
        xmin, ymin = reg['polygon'].min(axis=0)
        n_reg['topcorner'] = (xmin, ymin)
        regions.append(n_reg)
    s_regions = sorted(regions, key=lambda x: (x['topcorner'][1], x['topcorner'][0]), reverse=False)
    if len(page['regions']) == 3:
        
        print("Table title", page['regions'][s_regions[0]['index']]['polygon'])
        print("Table header", page['regions'][s_regions[1]['index']]['polygon'])
        for cell in page['regions'][s_regions[2]['index']]['table']:
            print("Cell coords", cell['polygon'])
    elif len(page['regions']) == 4:
        print("Table header", page['regions'][s_regions[0]['index']]['polygon'])
        print("Table header", page['regions'][s_regions[1]['index']]['polygon'])
        for cell in page['regions'][s_regions[2]['index']]['table']:
            print("Cell coords", cell['polygon'])
        for cell in page['regions'][s_regions[3]['index']]['table']:
            print("Cell coords", cell['polygon'])
    elif len(page['regions']) == 6:
        print("Table title", page['regions'][s_regions[0]['index']]['polygon'])
        print("Table title", page['regions'][s_regions[1]['index']]['polygon'])
        print("Table header", page['regions'][s_regions[2]['index']]['polygon'])
        print("Table header", page['regions'][s_regions[3]['index']]['polygon'])
        for cell in page['regions'][s_regions[4]['index']]['table']:
            print("Cell coords", cell['polygon'])
        for cell in page['regions'][s_regions[5]['index']]['table']:
            print("Cell coords", cell['polygon'])
    else:
        print("Annotation error", idx, dataset.__getname__(idx))

In [ ]:
regions

In [ ]:
 sorted(regions, key=lambda x: (x['topcorner'][1], x['topcorner'][0]), reverse=False)

In [ ]:
def get_cell_corners(page):
    cell_corners = {}

    for cell in reg['table']:
        row = cell['row']
        col = cell['col']
        xmax, ymax = cell['polygon'].max(axis=0)
        xmin, ymin = cell['polygon'].min(axis=0)
        top_left = (xmin, ymin)
        top_right = (xmax, ymin)
        bottom_left = (xmin, ymax)
        bottom_right = (xmax, ymax)
        cell_corners[(row, col)] = [top_left, top_right, bottom_right, bottom_left]

    return cell_corners



In [ ]:
for i, reg in enumerate(page['regions']):
    if reg['regiontype'] == "TableRegion":
        print("Table:", i)
        cell_corners = get_cell_corners(reg['table'])
        for key, corners in cell_corners.items():
            print(f"Cell {key} corners: {corners}")


In [ ]:
idx